In [3]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import numpy as np

# Load the train and test datasets
train_data = pd.read_csv('dataset/train1.csv')

test_data = pd.read_csv('dataset/sample_test.csv')

# Extract the image URLs from the dataset
image_urls_train = train_data['image_link']
image_urls_test = test_data['image_link']

# Function to fetch images directly from the URL
def fetch_image_from_url(image_url):
    """Fetch image from URL, preprocess it, and return it as a numpy array."""
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img = img.resize((224, 224))  # Resize image for ResNet50 input
        img_array = np.array(img) / 255.0  # Normalize image to [0, 1]
        return img_array
    except Exception as e:
        print(f"Error fetching image {image_url}: {e}")
        return None


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

# Initialize ResNet50 model with pretrained weights
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def extract_image_features(image_url):
    """Extract features from an image URL using ResNet50."""
    img_array = fetch_image_from_url(image_url)
    if img_array is None:
        return None
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    
    features = resnet_model.predict(img_array)
    return features.flatten()  # Flatten the features


In [5]:
# Extract features from training images directly from URLs
train_features = []
for i, url in enumerate(image_urls_train):
    features = extract_image_features(url)
    if features is not None:
        train_features.append(features)

train_features = np.array(train_features)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re  # To help with cleaning numeric values
import pandas as pd
import numpy as np

# Function to split entity_value into numeric value and unit
def split_value_unit(entity_value):
    if pd.isna(entity_value):
        return None, None
    try:
        # Remove any square brackets or other unwanted characters
        cleaned_value = re.sub(r'[\[\],]', '', entity_value).strip()
        parts = cleaned_value.split()
        value = float(parts[0])  # Convert first part to float (assuming it is the numeric value)
        unit = ' '.join(parts[1:])  # Remaining part is considered the unit
        return value, unit
    except (ValueError, IndexError):
        print(f"Warning: Could not convert {entity_value} to float.")
        return None, None

# Apply the function to split entity_value into numeric and unit
train_data['value'], train_data['unit'] = zip(*train_data['entity_value'].map(split_value_unit))

# Drop rows where 'value' or 'unit' is None due to conversion issues
train_data_cleaned = train_data.dropna(subset=['value', 'unit']).reset_index(drop=True)

# Convert train_features to a numpy array if it is not already
if isinstance(train_features, pd.DataFrame):
    train_features = train_features.to_numpy()

# Align the length of train_features to match the cleaned data
# Ensure we slice train_features to match the length of train_data_cleaned
train_features_cleaned = train_features[:len(train_data_cleaned)]

# Encode the units
unit_encoder = LabelEncoder()
train_data_cleaned['unit_encoded'] = unit_encoder.fit_transform(train_data_cleaned['unit'])

# Prepare the target values for regression and classification
y_numeric = train_data_cleaned['value'].values
y_unit = train_data_cleaned['unit_encoded'].values

# Check lengths to debug
# print(f"Length of train_features_cleaned: {len(train_features_cleaned)}")
# print(f"Length of y_numeric: {len(y_numeric)}")
# print(f"Length of y_unit: {len(y_unit)}")

#delete the last record from y_numeric and y_unit to match the length of train_features_cleaned
y_numeric = y_numeric[:-1]
y_unit = y_unit[:-1]

#check the lengths again
# print(f"Length of train_features_cleaned: {len(train_features_cleaned)}")
# print(f"Length of y_numeric: {len(y_numeric)}")
# print(f"Length of y_unit: {len(y_unit)}")

# # Ensure all lengths match
assert len(train_features_cleaned) == len(y_numeric) == len(y_unit), "Mismatch between feature and target lengths."

# # Train-test split
X_train, X_val, y_train_numeric, y_val_numeric, y_train_unit, y_val_unit = train_test_split(
    train_features_cleaned, y_numeric, y_unit, test_size=0.2, random_state=42
)


In [8]:
from tensorflow.keras import layers, models

# Define the multi-output model
def build_model():
    input_layer = layers.Input(shape=(train_features.shape[1],))
    
    # Common layers
    dense1 = layers.Dense(128, activation='relu')(input_layer)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    
    # Numeric regression output
    output_numeric = layers.Dense(1, name='numeric_output')(dense2)
    
    # Unit classification output
    output_unit = layers.Dense(len(unit_encoder.classes_), activation='softmax', name='unit_output')(dense2)
    
    model = models.Model(inputs=input_layer, outputs=[output_numeric, output_unit])
    
    model.compile(optimizer='adam',
                  loss={'numeric_output': 'mean_squared_error', 'unit_output': 'sparse_categorical_crossentropy'},
                  metrics={'numeric_output': 'mae', 'unit_output': 'accuracy'})
    return model

model = build_model()

# Display model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100352)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 12,845,184 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numeric_output      │ (None, 1)         │         65 │ dense_1[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ unit_output (Dense) │ (None, 92)        │      5,980 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,859,485 (49.06 MB)

 Trainable params: 12,859,485 (49.06 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history = model.fit(
    X_train, 
    {'numeric_output': y_train_numeric, 'unit_output': y_train_unit},
    validation_data=(X_val, {'numeric_output': y_val_numeric, 'unit_output': y_val_unit}),
    epochs=10,
    batch_size=32
)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 111ms/step - loss: 1696672203669504.0000 - numeric_output_loss: 1696514766274560.0000 - numeric_output_mae: 367482.4375 - unit_output_accuracy: 0.1120 - unit_output_loss: 24.9542 - val_loss: inf - val_numeric_output_loss: inf - val_numeric_output_mae: 222233324565823488.0000 - val_unit_output_accuracy: 0.5050 - val_unit_output_loss: 9.4860
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 101ms/step - loss: 31406561804943360.0000 - numeric_output_loss: 31406402891153408.0000 - numeric_output_mae: 3628766.7500 - unit_output_accuracy: 0.2690 - unit_output_loss: 10.5937 - val_loss: inf - val_numeric_output_loss: inf - val_numeric_output_mae: 222233324565823488.0000 - val_unit_output_accuracy: 0.0930 - val_unit_output_loss: 23.7966
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 106ms/step - loss: 24936799163908096.0000 - numeric_output_loss: 24936642397601792.0000 - numeric_output_mae: 2939092.2500 - unit_output_accuracy: 0.2663 - unit_output_loss: 17.008

In [10]:
# Extract features for test data
test_features = []
for i, url in enumerate(image_urls_test):
    features = extract_image_features(url)
    if features is not None:
        test_features.append(features)

test_features = np.array(test_features)

# Make predictions
pred_numeric, pred_unit = model.predict(test_features)

# Convert predictions to the required format
pred_unit_labels = unit_encoder.inverse_transform(np.argmax(pred_unit, axis=1))
predictions = [f"{round(pred_val, 2)} {unit}" for pred_val, unit in zip(pred_numeric.flatten(), pred_unit_labels)]

# Prepare the submission file
submission = pd.DataFrame({'index': test_data['index'], 'prediction': predictions})
submission.to_csv('test_output.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━